# Capella API: Search, Order, and View Tutorial

In [ ]:
# Required libraries:
# requests
# json
# folium
# rasterio
# from rasterio.plot import show
# from rasterio.windows import Window
# from skimage import exposure
# from matplotlib import pyplot as plt

In [ ]:
!pip install folium rasterio 

Your username and password must be saved in a .json file named 'credentials.json' and formatted as follows.

{"username": "yourusername","password": "xxxxxxxxx"}

### Import required libraries, build a print utility function, assign API endpoints and load Credentials

In [ ]:
import requests
import json

# JSON utility function
def p(data):
    print(json.dumps(data, indent=2))

# Capella API endpoints
URL = 'https://api.capellaspace.com'
token = '/token'
collections = '/catalog/collections'
catsearch = '/catalog/search'
orders = '/orders/'

# Load username and password
with open('credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']

### Get and Print Access Token

In [ ]:
# Get token
r = requests.post(URL + token, 
                  headers = {'Content-Type': 'application/x-www-form-urlencoded'}, auth=(username,password))

accesstoken = r.json()["accessToken"]

# print the token
print("Access Token: " + accesstoken)

headers = {'Authorization':'Bearer ' + accesstoken}

### Print Available Collections

In [ ]:
# See what collections are available
r = requests.get(URL + collections, headers=headers)

# Print results
p(r.json())

### Post Search Filters, Print the Results

In [ ]:
# Post search filters
filters = {
  "datetime": "2019-01-01T00:00:00Z/2019-12-31T12:31:12Z", # start date/end date and time in YYYY-MM-DDTHH:MM:SSZ format
  "limit": 50, # overwrite the default pagination limit of 10, adjust as necessary
  "collections": ["capella-aerial"], # specify the desired collection
  "query": {"sar:product_type": {"eq":"GEO"}, # specify the type of product desired
            "sar:instrument_mode": {"eq":"spotlight"}}, # specify the instrument mode desired      
  "sort": [{"field": "dtr:start_datetime","direction": "desc"}],
}
headers = {'Content-Type': 'application/json',
  'Accept': 'application/geo+json', 'Authorization':'Bearer ' + accesstoken}
r = requests.post(URL + catsearch, json=filters, headers=headers)

# Inspect the results
p(r.json())

### View the Results of the Search

In [ ]:
# Display the results on a folium map
import folium

m = folium.Map(location=[37.84, -121.91],zoom_start=9)
folium.GeoJson(
    r.text,
).add_to(m)

m

### Make and Post an Order

In [ ]:
# Build the Order
features = r.json()["features"]
granulelist = []

# Loop over all the features from the response and add to an array for an order
for f in features:
    item = {"CollectionId": f["collection"], "GranuleId": f["id"]}
    granulelist.append(item)
    
myorder = {"Items": granulelist}

# Post the order and inspect the result
r = requests.post(URL + orders, json=myorder, headers=headers)
p(r.json())

### Get the STAC records with the signed URLs using the /download endpoint, Print the Result

In [ ]:
myorderid = r.json()["orderId"]
r = requests.get(URL + orders + myorderid + '/download', headers=headers)
p(r.json())

### Visualize the Results

In [ ]:
import matplotlib.pyplot as plt

#Open and inspect one of the images
features = r.json()
filepath = features[0]["assets"]["VV"]["href"]
#print(filepath)# print the url to download this asset

# Open the file with Rasterio
import rasterio
from rasterio.plot import show
from rasterio.windows import Window
from skimage import exposure
from matplotlib import pyplot as plt
Session = rasterio.Env()

with Session:
    with rasterio.open(filepath) as src:
        meta = src.meta
        tags = src.tags()
        #print(meta)
        #print(f"{'-' * 100}\n\n")
        w = meta.get("width")
        h = meta.get("height")
        w = Window(w/6, h/6, 2048, 2048)
        data = src.read(1, window=w)
        data[data == meta['nodata']] = 0
        equ = exposure.adjust_log(data, gain=2)
        fig, ax = plt.subplots(1, 1, figsize=(12,12))
        show(equ, cmap='gray')


              
    

